# ITI110 Project : Energy Consumption Forecasting (Hourly dataset)

## Data Preprocessing for Hourly Energy Consumption Dataset (Carel)

In [ ]:
import pandas as pd
import numpy as np
import tqdm
import os
import warnings
warnings.filterwarnings('ignore')
np.random.seed()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

### Load Data For Hourly Dataset

In [ ]:
df = pd.read_csv("2.2 merged_aggregated_hourly_without_block_id.csv")

# folder_path = r'C:\Users\Carel\Documents\Course NYP Specialist Diploma in Applied AI PDC2\SDAAI PDC2\ITI110 Project\code file\WIP project'
# csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# os.listdir(folder_path)


In [ ]:
# !pip install dask

In [ ]:
# !pip show pyarrow

In [ ]:
df.head()

,date,month,hour,count_LCLid,count_block_id,sum_energy_by_hour,tstp,avg_energy_per_lclid,holiday,visibility,...,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,2011-11-23,11,9,2,2,1.130,2011-11-23 09:00:00,0.565000,0.0,4.39,...,4.84,2011-11-23 09:00:00,4.68,1027.29,3.42,1.78,rain,partly-cloudy-day,0.99,Partly Cloudy
1,2011-11-23,11,10,6,4,1.508,2011-11-23 10:00:00,0.251333,0.0,6.24,...,5.74,2011-11-23 10:00:00,5.46,1027.67,4.24,1.99,rain,partly-cloudy-day,0.98,Partly Cloudy
2,2011-11-23,11,11,7,5,2.123,2011-11-23 11:00:00,0.303286,0.0,10.20,...,7.67,2011-11-23 11:00:00,5.74,1027.56,5.94,2.66,rain,partly-cloudy-day,0.88,Partly Cloudy
3,2011-11-23,11,12,10,7,6.589,2011-11-23 12:00:00,0.658900,0.0,11.62,...,8.84,2011-11-23 12:00:00,6.52,1027.24,7.25,2.80,rain,partly-cloudy-day,0.85,Partly Cloudy
4,2011-11-23,11,13,13,9,6.263,2011-11-23 13:00:00,0.481769,0.0,12.68,...,9.87,2011-11-23 13:00:00,6.99,1027.35,8.29,3.12,rain,partly-cloudy-day,0.82,Mostly Cloudy


In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
df.columns

Index(['date', 'month', 'hour', 'count_LCLid', 'count_block_id',
       'sum_energy_by_hour', 'tstp', 'avg_energy_per_lclid', 'holiday',
       'visibility', 'windBearing', 'temperature', 'time', 'dewPoint',
       'pressure', 'apparentTemperature', 'windSpeed', 'precipType', 'icon',
       'humidity', 'summary'],
      dtype='object')

In [ ]:
# Initial Overview of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19864 entries, 0 to 19863
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  19864 non-null  object 
 1   month                 19864 non-null  int64  
 2   hour                  19864 non-null  int64  
 3   count_LCLid           19864 non-null  int64  
 4   count_block_id        19864 non-null  int64  
 5   sum_energy_by_hour    19864 non-null  float64
 6   tstp                  19864 non-null  object 
 7   avg_energy_per_lclid  19864 non-null  float64
 8   holiday               19864 non-null  float64
 9   visibility            19862 non-null  float64
 10  windBearing           19862 non-null  float64
 11  temperature           19862 non-null  float64
 12  time                  19862 non-null  object 
 13  dewPoint              19862 non-null  float64
 14  pressure              19851 non-null  float64
 15  apparentTemperature

In [ ]:
# Compute missing values to view actual missing value count for each column
df.isnull().sum()

date                     0
month                    0
hour                     0
count_LCLid              0
count_block_id           0
sum_energy_by_hour       0
tstp                     0
avg_energy_per_lclid     0
holiday                  0
visibility               2
windBearing              2
temperature              2
time                     2
dewPoint                 2
pressure                13
apparentTemperature      2
windSpeed                2
precipType               2
icon                     2
humidity                 2
summary                  2
dtype: int64

#### Drop the category columns

In [ ]:
df = df.drop(['precipType', 'icon', 'summary'], axis=1)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'])
df['tstp'] = pd.to_datetime(df['tstp'])


In [ ]:
import pandas as pd

# Create a new DataFrame with just the 'date', 'tstp' and 'time'  columns
date_tstp_time_df = df[['date', 'tstp', 'time']]

# Display the new DataFrame
print(date_tstp_time_df)

            date                tstp                time
0     2011-11-23 2011-11-23 09:00:00 2011-11-23 09:00:00
1     2011-11-23 2011-11-23 10:00:00 2011-11-23 10:00:00
2     2011-11-23 2011-11-23 11:00:00 2011-11-23 11:00:00
3     2011-11-23 2011-11-23 12:00:00 2011-11-23 12:00:00
4     2011-11-23 2011-11-23 13:00:00 2011-11-23 13:00:00
...          ...                 ...                 ...
19859 2014-02-27 2014-02-27 20:00:00 2014-02-27 20:00:00
19860 2014-02-27 2014-02-27 21:00:00 2014-02-27 21:00:00
19861 2014-02-27 2014-02-27 22:00:00 2014-02-27 22:00:00
19862 2014-02-27 2014-02-27 23:00:00 2014-02-27 23:00:00
19863 2014-02-28 2014-02-28 00:00:00 2014-02-28 00:00:00

[19864 rows x 3 columns]


In [ ]:
# Check if the 'date' and 'time' columns are the same
are_same = (df['tstp'] == df['time']).all()

if are_same:
    print("The 'tstp' and 'time' columns have the same values.")
else:
    print("The 'tstp' and 'time' columns have different values.")


The 'tstp' and 'time' columns have different values.


#### Filter to tstp and time columns to see where the non-matches lie

In [ ]:
import pandas as pd

# Filter rows where 'tstp' and 'time' differ
filtered_df = df[df['tstp'] != df['time']]

# Display the differences
filtered_df[['date', 'tstp', 'time']]


,date,tstp,time
15758,2013-09-09,2013-09-09 23:00:00,NaT
15759,2013-09-10,2013-09-10 00:00:00,NaT


#### Notes: Since 'time' column is exact match with 'tstp' column, except for 2 rows: 15758 and 15759. It will not make sense to inpute the dates in these rows, therefore we will drop these rows.

In [ ]:
# Drop specific rows from the DataFrame
df = df.drop([15758, 15759])

In [ ]:
df.isnull().sum()

date                     0
month                    0
hour                     0
count_LCLid              0
count_block_id           0
sum_energy_by_hour       0
tstp                     0
avg_energy_per_lclid     0
holiday                  0
visibility               0
windBearing              0
temperature              0
time                     0
dewPoint                 0
pressure                11
apparentTemperature      0
windSpeed                0
humidity                 0
dtype: int64

#### Identify the missing data from pressure column

In [ ]:
missing_pressure = df['pressure'].isnull()

df_missing_pressure = df[missing_pressure]
df_missing_pressure

,date,month,hour,count_LCLid,count_block_id,sum_energy_by_hour,tstp,avg_energy_per_lclid,holiday,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,humidity
3993,2012-05-07,5,18,2348,112,1392.479999,2012-05-07 18:00:00,0.593049,0.0,11.27,189.0,12.99,2012-05-07 18:00:00,8.29,NaN,12.99,4.22,0.73
8118,2012-10-26,10,15,5494,112,2668.932000,2012-10-26 15:00:00,0.485790,0.0,11.27,21.0,7.29,2012-10-26 15:00:00,3.68,NaN,4.21,4.99,0.78
9766,2013-01-03,1,7,5486,112,1993.104000,2013-01-03 07:00:00,0.363307,0.0,9.66,250.0,9.65,2013-01-03 07:00:00,8.67,NaN,7.07,5.21,0.94
13540,2013-06-09,6,13,5319,112,2377.313000,2013-06-09 13:00:00,0.446947,0.0,9.98,48.0,13.42,2013-06-09 13:00:00,7.76,NaN,13.42,5.07,0.69
14799,2013-08-01,8,0,5268,112,1410.029000,2013-08-01 00:00:00,0.267659,0.0,9.98,193.0,19.16,2013-08-01 00:00:00,16.78,NaN,19.38,2.80,0.86
15160,2013-08-16,8,1,5241,112,1152.973000,2013-08-16 01:00:00,0.219991,0.0,9.98,217.0,17.01,2013-08-16 01:00:00,14.84,NaN,17.04,3.83,0.87
15398,2013-08-25,8,23,5221,112,1573.785000,2013-08-25 23:00:00,0.301434,0.0,6.39,10.0,17.56,2013-08-25 23:00:00,15.49,NaN,17.66,3.75,0.88
15399,2013-08-26,8,0,5227,112,1391.607001,2013-08-26 00:00:00,0.266234,1.0,4.39,358.0,17.02,2013-08-26 00:00:00,15.42,NaN,17.13,2.72,0.90
15736,2013-09-09,9,1,5216,112,1095.127000,2013-09-09 01:00:00,0.209955,0.0,9.51,221.0,9.39,2013-09-09 01:00:00,8.03,NaN,8.03,2.61,0.91
15739,2013-09-09,9,4,5216,112,1014.337000,2013-09-09 04:00:00,0.194466,0.0,9.46,224.0,9.99,2013-09-09 04:00:00,7.72,NaN,9.64,1.51,0.86


In [ ]:
df['pressure'] = df['pressure'].fillna(df['pressure'].mean())

# check that it's been filled
# df.isnull().sum()

In [ ]:
# # save updated dataset if desired
# data = df.to_csv('2.2 merged_aggregated_hourly_without_block_id_prepd.csv')

### Splitting the data to prepare for training

In [ ]:
import pandas as pd

train_rows = int(0.8*len(df))

train = df[:train_rows]
test = df[train_rows:]

# Check rows
print("dataset no. of rows: ", len(df))
print("train dataset no. of rows: ", len(train))
print("test dataset no. of rows: ", len(test))


# parameters for training subject to change depending on experiment plans
x_train = train[['temperature', 'humidity', 'windSpeed', 'holiday']]
y_train = train[['avg_energy_per_lclid']]

x_test = test[['temperature', 'humidity', 'windSpeed', 'holiday']]
y_test = test[['avg_energy_per_lclid']]

dataset no. of rows:  19862
train dataset no. of rows:  15889
test dataset no. of rows:  3973


### Normalization / Scaling of data

#### Notes: Normalizing and scaling will depend on the model chosen

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the training data and transform the training and test data
x_train_normalized = scaler.fit_transform(x_train)
x_test_normalized = scaler.transform(x_test)

# Now x_train_normalized and x_test_normalized contain the normalized values of the respective features


In [ ]:
# View the normalized data

print("Normalized x_train data:")
print(x_train_normalized[:10])

print("Normalized x_test data:")
print(x_test_normalized[:10])


Normalized x_train data:
[[0.27549947 0.98701299 0.13043478 0.        ]
 [0.29915878 0.97402597 0.14617691 0.        ]
 [0.34989485 0.84415584 0.1964018  0.        ]
 [0.38065195 0.80519481 0.20689655 0.        ]
 [0.40772871 0.76623377 0.23088456 0.        ]
 [0.42060988 0.79220779 0.24137931 0.        ]
 [0.41351209 0.80519481 0.22488756 0.        ]
 [0.40694006 0.81818182 0.21814093 0.        ]
 [0.38853838 0.8961039  0.1904048  0.        ]
 [0.38801262 0.87012987 0.21514243 0.        ]]
Normalized x_test data:
[[0.51025237 0.58441558 0.53898051 0.        ]
 [0.52260778 0.62337662 0.57946027 0.        ]
 [0.51840168 0.76623377 0.55247376 0.        ]
 [0.51182965 0.84415584 0.60044978 0.        ]
 [0.52129338 0.87012987 0.62068966 0.        ]
 [0.52260778 0.90909091 0.59970015 0.        ]
 [0.53785489 0.92207792 0.56521739 0.        ]
 [0.54416404 0.90909091 0.51274363 0.        ]
 [0.55205047 0.85714286 0.40704648 0.        ]
 [0.51524711 0.80519481 0.38755622 0.        ]]


### The decision was made as a group to leave room for flexibility in terms of normalization and standardization depending on the model chosen.